In [1]:
import os

# kaggle.json 파일을 열어서 보이는 username과 key를 입력해
os.environ['KAGGLE_USERNAME'] = "davidpark12"
os.environ['KAGGLE_KEY'] = "df63a35b59947510bcb44137aeb6e873"

# kaggle 라이브러리
import kaggle
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition
!unzip -q dogs-vs-cats-redux-kernels-edition.zip
!unzip -q train.zip -d.

 82% 669M/814M [00:00<00:00, 1.75GB/s]
100% 814M/814M [00:00<00:00, 1.74GB/s]


In [2]:
import os
import shutil

# 파일길이 보기
print(len(os.listdir('/content/train')))

# 폴더만들기
os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

# 고양이는 고양이로 강아지는 강아지로
for i in os.listdir('/content/train'):
    if 'cat' in i:
        shutil.copyfile('/content/train/' + i, '/content/dataset/cat/' + i)
    if 'dog' in i:
        shutil.copyfile('/content/train/' + i, '/content/dataset/dog/' + i)

25000


In [4]:
# 이미지 -> 숫자
import tensorflow as tf
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(299, 299),
    batch_size=64,
    subset='training',
    validation_split=0.2,
    seed=1234
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(299, 299),
    batch_size=64,
    subset='validation',
    validation_split=0.2,
    seed=1234
)

print(train_ds)


#잘되었는가 확인
# import matplotlib.pyplot as plt


# for i, 정답 in train_ds.take(1):
#     print(i)
#     print(정답)
#     plt.imshow(i[0].numpy().astype('uint8'))
#     plt.show()

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [5]:
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

open('inception_v3.h5', 'wb').write(r.content)

298

In [6]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

Inception_model = InceptionV3(
    input_shape=(299, 299, 3),
    include_top=False,
    weights="imagenet"
)

Inception_model.summary()

#학습금지레이어 설정
for i in Inception_model.layers:
    i.trainable = False

# 파인튜닝 해보기 'mixed6'이하 부터
ufreeze = False
for i in Inception_model.layers:
    if i.name == 'mixed6':
        ufreeze = True
    if ufreeze:
        i.trainable = True



#원하는 레이어 꺼내기
마지막레이어 = Inception_model.get_layer('mixed7')
display(마지막레이어)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 21,768,352 (83.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

<Concatenate name=mixed7, built=True>

In [9]:
# 내 레이어 만들기
import tensorflow as tf

layer1 = tf.keras.layers.Flatten()(마지막레이어.output)
layer2 = tf.keras.layers.Dense(1024, activation='relu')(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
layer3 = tf.keras.layers.Dense(1, activation='sigmoid')(drop1)

#어디가 시작이고 끝인지 알려주기
model = tf.keras.Model(Inception_model.input, layer3)

model.compile( loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), metrics=['acc'] )
model.fit(train_ds, validation_data = val_ds, epochs=5)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 43s 92ms/step - acc: 0.7079 - loss: 0.5589 - val_acc: 0.7588 - val_loss: 0.5006
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 50ms/step - acc: 0.8341 - loss: 0.3732 - val_acc: 0.7736 - val_loss: 0.4848
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 50ms/step - acc: 0.8947 - loss: 0.2622 - val_acc: 0.7790 - val_loss: 0.5000
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 50ms/step - acc: 0.9420 - loss: 0.1764 - val_acc: 0.7834 - val_loss: 0.5095
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 50ms/step - acc: 0.9719 - loss: 0.1151 - val_acc: 0.7864 - val_loss: 0.5541
